In [1]:
import pandas as pd

In [10]:
# Load the cbp-landsat dataset
path = 'IOP-landsat-matchup.csv'
df_chla = pd.read_csv(path)
df_chla.columns = df_chla.columns.str.strip()
df_chla = df_chla.iloc[0:2,:]
df_chla.head()

,Unnamed: 0.1,Unnamed: 0,pressure,sal,Wt,CHL,agp400.9,agp404.3,agp407.8,agp410.9,...,coastal_n,blue_n,green_n,red_n,nir08_n,datetime,eo:cloud_cover,proj:epsg,view:sun_azimuth,view:sun_elevation
0,0,0,1.5,0.483175,25.087299,9.128,5.131305,5.028419,4.901094,4.786908,...,81,81,81,81,81,2023-03-16T15:46:47.795568Z,0.03,32618,146.453734,45.525066
1,1,1,1.5,6.943292,25.274099,5.593,2.525151,2.443801,2.379273,2.294137,...,81,81,81,81,81,2023-04-26T15:40:10.710784Z,15.98,32618,137.686388,60.083081


## PCA

In [11]:
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Define the features X and the target variable y
ref = np.stack([df_chla['coastal'], df_chla['blue'], df_chla['green'],
             df_chla['red'], df_chla['nir08']], axis=1)

# Load the pre-fitted PCA model
pca = load('PCA.joblib')
# Apply the PCA transformation to your data
ref_pca = pca.transform(ref)
scores = ref_pca[:]
X = np.hstack((scores, scores**2))
X = sm.add_constant(X)

In [12]:
pred_chla = 2.5950833611051665 + (68.77680402403622*X[:,2]) + (501.1871737936483*X[:,5]) - (1504.4350127079304*X[:,7]) - (229935.4547152554*X[:,10]) - (16.681559328601512*X[:,3]) - (33.68307477430513*X[:,6])
pred_chla

array([1.98210327, 2.2197632 ])

In [13]:
df_chla['pred_logchl'] = pred_chla
df_chla

,Unnamed: 0.1,Unnamed: 0,pressure,sal,Wt,CHL,agp400.9,agp404.3,agp407.8,agp410.9,...,blue_n,green_n,red_n,nir08_n,datetime,eo:cloud_cover,proj:epsg,view:sun_azimuth,view:sun_elevation,pred_logchl
0,0,0,1.5,0.483175,25.087299,9.128,5.131305,5.028419,4.901094,4.786908,...,81,81,81,81,2023-03-16T15:46:47.795568Z,0.03,32618,146.453734,45.525066,1.982103
1,1,1,1.5,6.943292,25.274099,5.593,2.525151,2.443801,2.379273,2.294137,...,81,81,81,81,2023-04-26T15:40:10.710784Z,15.98,32618,137.686388,60.083081,2.219763


In [14]:
df_chla['CHLA'] = np.exp(df_chla['pred_logchl'])
df_chla

,Unnamed: 0.1,Unnamed: 0,pressure,sal,Wt,CHL,agp400.9,agp404.3,agp407.8,agp410.9,...,green_n,red_n,nir08_n,datetime,eo:cloud_cover,proj:epsg,view:sun_azimuth,view:sun_elevation,pred_logchl,CHLA
0,0,0,1.5,0.483175,25.087299,9.128,5.131305,5.028419,4.901094,4.786908,...,81,81,81,2023-03-16T15:46:47.795568Z,0.03,32618,146.453734,45.525066,1.982103,7.257992
1,1,1,1.5,6.943292,25.274099,5.593,2.525151,2.443801,2.379273,2.294137,...,81,81,81,2023-04-26T15:40:10.710784Z,15.98,32618,137.686388,60.083081,2.219763,9.205151
